In [49]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [50]:
dfTest = pd.read_csv('C:/Users/janine van wonderen/Desktop/dmt2/test_set_VU_DM.csv',usecols=['srch_id','prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price','srch_destination_id','orig_destination_distance'])
dfTrain = pd.read_csv('C:/Users/janine van wonderen/Desktop/dmt2/training_set_VU_DM.csv',usecols=['srch_id','prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price','srch_destination_id','booking_bool','orig_destination_distance'])

In [51]:
together = pd.concat([dfTest,dfTrain],axis=0)

C:\Users\janine van wonderen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [52]:
together['mean_distance'] = together.groupby(['srch_id'])['orig_destination_distance'].mean()
together['diff_distance_orig'] = (together['orig_destination_distance'] - together['mean_distance'])
srchIdMeans= together.groupby(['srch_id']).mean().reset_index()

In [53]:
columnsToAvgPerPropID = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price']
groupedMean = together.groupby('prop_id')[columnsToAvgPerPropID].mean()
groupedMean = groupedMean.add_prefix("mean_").reset_index()
groupedMean.to_csv("meansTotal.csv",index=False)

In [54]:
groupedProp = dfTrain.groupby(['srch_destination_id','prop_id']).sum().reset_index()
groupedProp = groupedProp.rename(columns = {"booking_bool": "bookingSum"})

In [55]:
def joinPropDest(df):
    df = df.merge(right=groupedProp[['srch_destination_id','prop_id','bookingSum']],how="left",on=['srch_destination_id','prop_id'],validate="many_to_one")
    return df

In [56]:
def joinDistance(df):
    df = df.merge(right=srchIdMeans[['srch_id','orig_destination_distance']],how="left",on=['srch_id'],validate="many_to_one")
    return df

In [57]:
def joinMean(df):
    df = df.merge(right=meanDF,how="left",on="prop_id",validate="many_to_one")
    return df

In [58]:
meanDF= pd.read_csv("meansTotal.csv")

In [59]:
mms = MinMaxScaler()
data = meanDF.drop(['prop_id','mean_prop_review_score','mean_prop_location_score2'],axis=1)
print(data.columns)
mms.fit(data)
data_transformed = mms.transform(data)
km = KMeans(n_clusters=6)
km = km.fit(data_transformed)

Index(['mean_prop_starrating', 'mean_prop_brand_bool',
       'mean_prop_location_score1', 'mean_prop_log_historical_price'],
      dtype='object')


In [60]:
dummiesCluster = pd.get_dummies(km.labels_,prefix = "cluster_")

In [61]:
meanDF = pd.concat([meanDF,dummiesCluster],axis=1)

In [62]:
df = pd.read_csv('C:/Users/janine van wonderen/Desktop/dmt2/test_set_VU_DM.csv')

In [63]:
df = joinMean(df)

In [64]:
df = joinPropDest(df)

In [65]:
df = joinDistance(df)

In [66]:
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,mean_prop_location_score2,mean_prop_log_historical_price,cluster__0,cluster__1,cluster__2,cluster__3,cluster__4,cluster__5,bookingSum,orig_destination_distance_y
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,...,0.043491,4.196765,1,0,0,0,0,0,7.0,NaN
1,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,5543,3,4.5,...,0.038567,4.283044,1,0,0,0,0,0,11.0,NaN
2,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,14142,2,3.5,...,0.036760,3.786120,0,0,0,0,0,1,0.0,NaN
3,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,22393,3,4.5,...,0.037828,4.295411,1,0,0,0,0,0,2.0,NaN
4,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,24194,3,4.5,...,0.125222,4.011241,1,0,0,0,0,0,1.0,NaN


In [67]:
df.to_csv('C:/Users/janine van wonderen/Desktop/dmt2/testWithMean.csv',index=False)

In [68]:
df = pd.read_csv('C:/Users/janine van wonderen/Desktop/dmt2/training_set_VU_DM.csv')
df = joinMean(df)
df = joinPropDest(df)
df = joinDistance(df)
df.to_csv('C:/Users/janine van wonderen/Desktop/dmt2/trainWithMean.csv',index=False)

Split train in train and test

In [69]:
df = shuffle(df,random_state = 10)

In [70]:
n = 20
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
df.drop(df.tail(numOfRowsTest).index,inplace=True)

In [71]:
X_test.to_csv('C:/Users/janine van wonderen/Desktop/dmt2/20PercentTestWithMean.csv',index=False)
df.to_csv('C:/Users/janine van wonderen/Desktop/dmt2/80PercentTrainWithMean.csv',index=False)